In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import layers, Sequential,losses, metrics

image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness',
                  'anger', 'disgust', 'fear', 'contempt']

samples = 35393 # 2~35394
training_samples = 28317  # 2~28318 (Training)
validation_samples = 3541 # 28319~31859 (PublicTest)
test_samples = 3535       # 31860~35394 (PrivateTest)
expw_samples = 35000

image_path = "./dataset/images.npy"
emotion_path = "./dataset/emotions_single.npy"
image_path_expw = "./sample35k/images.npy"
emotion_path_expw = "./sample35k/emotions.npy"

In [ ]:
images = np.load(image_path)
emotions = np.load(emotion_path)
images_expw = np.load(image_path_expw)
emotions_expw = np.load(emotion_path_expw)

print(images.shape)
print(emotions.shape)
print(images_expw.shape)
print(emotions_expw.shape)

In [ ]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [ ]:
images = tf.convert_to_tensor(images)
images = tf.image.grayscale_to_rgb(images)
emotions = tf.convert_to_tensor(emotions)

In [ ]:
images_expw = tf.convert_to_tensor(images_expw)
emotions_expw = tf.convert_to_tensor(emotions_expw)

In [ ]:
images = layers.Rescaling(1./127.5, offset= -1)(images)
#images_expw = layers.Rescaling(1./127.5, offset= -1)(images_expw)

In [ ]:
training_size = training_samples + validation_samples
test_size = test_samples

In [ ]:
training_images = tf.concat([images_expw, images[:training_size]], 0)
test_images = images[training_size:]
training_emotions = tf.concat([emotions_expw, emotions[:training_size]], 0)
test_emotions = emotions[training_size:]

In [ ]:
samples += expw_samples
print("total sample:", samples)
print("training_images shape:", training_images.shape)
print("training_emotions shape:", training_emotions.shape)
print("test_images shape:", test_images.shape)
print("test_emotions shape:", test_emotions.shape)

In [ ]:
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import layers, Sequential,losses, metrics
from tensorflow.python.keras.applications import vgg16, resnet_v2
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.optimizer_v2 import adam

base_model = vgg16.VGG16(include_top=False, 
                         weights="imagenet", 
                         input_shape=(48,48,3))
base_model.trainable=True
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(emotions_count, activation='softmax'),
])

model.compile(optimizer=adam.Adam(learning_rate=1e-4), 
              loss=losses.CategoricalCrossentropy(), 
              metrics = [model_acc])

model.fit(x=training_images,
          y=training_emotions,
          batch_size=32,
          epochs=40,
          validation_data=(test_images, test_emotions))

In [ ]:
base_model = vgg16.VGG16(include_top=False, 
                         weights="imagenet", 
                         input_shape=(48,48,3))
base_model.trainable=True
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(emotions_count, activation='softmax'),
])

model.compile(optimizer='sgd', 
              loss='mse', 
              metrics = [model_acc])

model.fit(x=training_images,
          y=training_emotions,
          batch_size=32,
          epochs=20,
          validation_data=(test_images, test_emotions))